In [49]:
"""
FINAL END-TO-END CHURN PREDICTION & TIME-BASED EARLY WARNING SYSTEM
- No leakage
- Generalized
- Realistic
- Production-style evaluation
"""

import pandas as pd
import numpy as np
import shap
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE



# ======================================================
# NEW: ADVANCED FEATURE ENGINEERING (TRENDS)
# ======================================================
def engineer_trend_features(df):
    print("⚙း Engineering time-based trend features...")
    # Sort to ensure we compare the right months
    df = df.sort_values(by=['Partner_ID', 'snapshot_date'])

    # Columns we want to track changes for
    cols_to_track = ['Login_Freq_Last_30d', 'Referral_Volume_30d', 'Commission_Payout_Last_30d']

    for col in cols_to_track:
        # Calculate % Change from previous row
        # shift(1) gets the previous month's data for that partner
        prev_col = df.groupby('Partner_ID')[col].shift(1)

        # Create the Trend Column
        # (Current - Previous) / Previous
        df[f'{col}_Trend'] = (df[col] - prev_col) / (prev_col + 1e-6) # +1e-6 prevents divide by zero

        # If it's the first month, Trend is 0
        df[f'{col}_Trend'] = df[f'{col}_Trend'].fillna(0)

    return df

# ======================================================
# 1. CONFIGURATION (ONLY CHANGE THESE IF NEEDED)
# ======================================================

DATA_PATH = "/content/AI_Partner_Churn_Dataset.csv"
TARGET_COLUMN = "Churn_Label"
TIME_COLUMN = "snapshot_date"

EARLY_WARNING_THRESHOLD = 0.65
TRAIN_SPLIT_RATIO = 0.7
RANDOM_STATE = 42

# ======================================================
# 2. LOAD DATA
# ======================================================

df = pd.read_csv(DATA_PATH)

# ------------------------------------------------------
# Add time column if not present (CRITICAL: DO THIS BEFORE TRENDS)
# ------------------------------------------------------

if TIME_COLUMN not in df.columns:
    df[TIME_COLUMN] = pd.date_range(
        start="2023-01-01",
        periods=len(df),
        freq="D"
    )

# APPLY THE NEW ENGINEERING
df = engineer_trend_features(df)
print("Dataset shape:", df.shape)

# ======================================================
# 3. IDENTIFY ID & LEAKAGE COLUMNS
# ======================================================

LEAKAGE_COLS = [
    TARGET_COLUMN,
    "Partner_ID",
    "churn_probability",
    "Churn_Probability",
    "EarlyWarningScore",
    "Top_Risk_Drivers",
    "Recommended_Action",
    "risk_tier",
    "early_warning_flag",
    "Manager_Priority",
    "scenario_trigger"
]

# ======================================================
# 4. SORT BY TIME (CRITICAL FOR EARLY WARNING)
# ======================================================

df = df.sort_values(TIME_COLUMN).reset_index(drop=True)

# ======================================================
# 5. TIME-BASED TRAIN / TEST SPLIT
# ======================================================

split_index = int(len(df) * TRAIN_SPLIT_RATIO)

train_df = df.iloc[:split_index]
test_df  = df.iloc[split_index:]

# Drop leakage and ID columns from training features
cols_to_drop = [c for c in LEAKAGE_COLS + [TIME_COLUMN] if c in df.columns]

X_train = train_df.drop(columns=cols_to_drop)
y_train = train_df[TARGET_COLUMN]

X_test  = test_df.drop(columns=cols_to_drop)
y_test  = test_df[TARGET_COLUMN]

# ======================================================
# 6. AUTOMATIC FEATURE TYPE DETECTION
# ======================================================

cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical columns:", cat_cols)
print("Numerical columns:", num_cols)

# ======================================================
# 7. HANDLE MISSING VALUES
# ======================================================

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_test[num_cols]  = num_imputer.transform(X_test[num_cols])

X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])
X_test[cat_cols]  = cat_imputer.transform(X_test[cat_cols])

# ======================================================
# 8. ENCODE CATEGORICAL FEATURES
# ======================================================

encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))
    encoders[col] = le

# ======================================================
# 9. SCALE NUMERICAL FEATURES
# ======================================================

scaler = RobustScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# ======================================================
# 10. HANDLE CLASS IMBALANCE (TRAINING ONLY)
# ======================================================

print("\nClass distribution before SMOTE:")
print(y_train.value_counts())

smote = SMOTE(sampling_strategy=0.8, random_state=RANDOM_STATE)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

print("\nClass distribution after SMOTE:")
print(y_train_bal.value_counts())

# ======================================================
# 11. TRAIN XGBOOST MODEL
# ======================================================

model = XGBClassifier(
    n_estimators=600,
    max_depth=5,
    learning_rate=0.04,
    subsample=0.85,
    colsample_bytree=0.85,
    eval_metric="aucpr",   # IMPORTANT
    random_state=42
)

model.fit(X_train_bal, y_train_bal)

# ======================================================
# 12. TIME-BASED MODEL EVALUATION
# ======================================================

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\nTIME-BASED MODEL PERFORMANCE")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob), 4))

# ======================================================
# 13. EARLY WARNING DETECTION (HYBRID STRATEGY)
# ======================================================

test_df["churn_probability"] = y_prob

test_df["early_warning_flag"] = (
    test_df["churn_probability"] >= EARLY_WARNING_THRESHOLD
).astype(int)

actual_churns = test_df[test_df[TARGET_COLUMN] == 1]
early_detected = actual_churns[
    actual_churns["early_warning_flag"] == 1
]

early_detection_rate = len(early_detected) / len(actual_churns) if len(actual_churns) > 0 else 0

print("\nEARLY WARNING RESULTS")
print("Total future churns:", len(actual_churns))
print("Caught early:", len(early_detected))
print("Early detection rate:", round(early_detection_rate, 2))


# ======================================================
# 14. RISK TIERS
# ======================================================

def risk_tier(p):
    if p >= 0.75:
        return "HIGH"
    elif p >= 0.45:
        return "MEDIUM"
    return "LOW"

test_df["risk_tier"] = test_df["churn_probability"].apply(risk_tier)

# ======================================================
# 15. EXPLAINABILITY (SHAP)
# ======================================================

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

feature_names = X_test.columns.tolist()

def top_drivers_shap(shap_row, n=3):
    idx = np.argsort(np.abs(shap_row))[-n:]
    return ", ".join(feature_names[i] for i in idx)

test_df["top_churn_drivers"] = [
    top_drivers_shap(shap_values[i])
    for i in range(len(shap_values))
]


# Priority score = risk & value
# Using test_df because it contains the 'churn_probability' predictions
test_df["PriorityScore"] = (
    test_df["churn_probability"] *
    test_df["Commission_Payout_Last_30d"]
)

test_df["Manager_Priority"] = pd.qcut(
    test_df["PriorityScore"],
    q=[0, 0.7, 0.9, 1.0],
    labels=["Low", "Medium", "High"]
)

print("Priority Distribution:")
print(test_df["Manager_Priority"].value_counts())


cohort_risk = (
    test_df
    .groupby(["Region", "Partner_Tier"])
    .agg(
        partners=("Region", "count"),
        avg_churn_prob=("churn_probability", "mean"),
        high_risk_pct=("risk_tier", lambda x: (x == "HIGH").mean())
    )
    .reset_index()
)

print("Cohort Risk Analysis:")
print(cohort_risk.sort_values("avg_churn_prob", ascending=False).head())

def recommend_action(drivers):
    if "risk_no_contact" in drivers:
        return "Schedule immediate check-in call"
    if "risk_negative_sentiment" in drivers:
        return "Resolve support issue and follow up"
    if "risk_referral_drop" in drivers:
        return "Offer incentive or performance review"
    return "Monitor closely"

# ======================================================
# 16. SAVE FINAL OUTPUT
# ======================================================

test_df.to_csv("final_churn_time_based_predictions.csv", index=False)

print("\nSaved: final_churn_time_based_predictions.csv")

print("\nSample HIGH-RISK EARLY WARNINGS:")
print(
    test_df[test_df["risk_tier" ] == "HIGH"]
    [["churn_probability", "risk_tier", "early_warning_flag", "top_churn_drivers"]]
    .head()
)


⚙း Engineering time-based trend features...
Dataset shape: (1000, 22)
Categorical columns: ['Region', 'Partner_Tier']
Numerical columns: ['Years_In_Network', 'Login_Freq_Last_30d', 'Login_Trend_Index', 'Dashboard_Usage_Minutes', 'Referral_Volume_30d', 'Referral_Growth_Rate', 'Support_Tickets_Open', 'Recent_Feedback_Sentiment', 'Days_Since_Last_Contact', 'Commission_Payout_Last_30d', 'Unresolved_Disputes_Count', 'Competitor_Mentioned', 'Technical_Integration_Issues', 'Login_Freq_Last_30d_Trend', 'Referral_Volume_30d_Trend', 'Commission_Payout_Last_30d_Trend']

Class distribution before SMOTE:
Churn_Label
0    591
1    109
Name: count, dtype: int64

Class distribution after SMOTE:
Churn_Label
0    591
1    472
Name: count, dtype: int64

TIME-BASED MODEL PERFORMANCE
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       254
           1       0.60      0.52      0.56        46

    accuracy                           0.87       300
   macro

In [50]:
import pandas as pd

# ----------------------------------------
# LOAD LAST MODEL OUTPUT
# ----------------------------------------

df = pd.read_csv("final_churn_time_based_predictions.csv")

# ----------------------------------------
# SAFETY: NORMALIZE COLUMN NAMES
# ----------------------------------------

df.columns = [c.strip() for c in df.columns]

# ----------------------------------------
# FIX MISSING COLUMNS / NAME MISMATCHES
# ----------------------------------------

# 1. Fallback for Partner_ID (it was dropped to prevent leakage earlier)
if "Partner_ID" not in df.columns:
    df["Partner_ID"] = [f"PTR-{i+1000}" for i in range(len(df))]

# 2. Map 'top_churn_drivers' (from SHAP) to 'Top_Risk_Drivers'
if "top_churn_drivers" in df.columns:
    df["Top_Risk_Drivers"] = df["top_churn_drivers"]

# ----------------------------------------
# ENSURE REQUIRED COLUMNS EXIST
# ----------------------------------------

required_cols = [
    "Partner_ID",
    "Region",
    "Partner_Tier",
    "churn_probability",
    "risk_tier",
    "early_warning_flag",
    "Manager_Priority",
    "Top_Risk_Drivers"
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}. Available: {df.columns.tolist()}")

# ----------------------------------------
# RECOMMENDED ACTION LOGIC (ROBUST)
# ----------------------------------------

def recommend_action(drivers):
    drivers = str(drivers).lower()
    if "no_contact" in drivers or "days_since_last_contact" in drivers:
        return "Schedule immediate check-in call"
    if "negative_sentiment" in drivers or "feedback" in drivers:
        return "Resolve support issue and follow up"
    if "referral" in drivers:
        return "Offer incentive or performance review"
    if "login" in drivers:
        return "Re-engagement training or onboarding support"
    if "support_tickets" in drivers:
        return "Address technical hurdles or support backlog"
    return "Monitor closely"

df["Recommended_Action"] = df["Top_Risk_Drivers"].apply(recommend_action)

# ----------------------------------------
# FINAL STREAMLIT DATAFRAME
# ----------------------------------------

final_df = df[
    [
        "Partner_ID",
        "Region",
        "Partner_Tier",
        "churn_probability",
        "risk_tier",
        "early_warning_flag",
        "Manager_Priority",
        "Top_Risk_Drivers",
        "Recommended_Action"
    ]
].sort_values("churn_probability", ascending=False)

# ----------------------------------------
# SAVE CLEAN CSV
# ----------------------------------------

final_df.to_csv("streamlit_ready_churn_dashboard.csv", index=False)

print("✅ Clean Streamlit-ready CSV saved: streamlit_ready_churn_dashboard.csv")
print(f"Processed {len(final_df)} partners.")
print(final_df.head())

✅ Clean Streamlit-ready CSV saved: streamlit_ready_churn_dashboard.csv
Processed 300 partners.
       Partner_ID Region Partner_Tier  churn_probability risk_tier  \
21   PARTNER_0721   AMER       Silver           0.995443      HIGH   
171  PARTNER_0871  LATAM       Silver           0.985463      HIGH   
54   PARTNER_0754   APAC       Bronze           0.982124      HIGH   
77   PARTNER_0777  LATAM       Bronze           0.969371      HIGH   
144  PARTNER_0844   EMEA       Bronze           0.967789      HIGH   

     early_warning_flag Manager_Priority  \
21                    1             High   
171                   1             High   
54                    1             High   
77                    1             High   
144                   1             High   

                                      Top_Risk_Drivers  \
21   Days_Since_Last_Contact, Support_Tickets_Open,...   
171  Competitor_Mentioned, Support_Tickets_Open, Re...   
54   Recent_Feedback_Sentiment, Days_Since_La